In [281]:
!pip install openai==1.2.0
import json

In [282]:
data = {

    "기본정보": "안녕하세요! 바닷가앞펜션은 애월 해안도로 제일 높은 곳에 위치해 있어 탁 트인 바다 전망을 감상하실 수 있습니다. 펜션 앞 도로는 제주 환상길 산책로가 있어 자연 경관을 즐기며 산책하기 좋습니다. 다양한 콘셉트의 깨끗한 객실과 함께 가족, 연인 모두가 즐길 수 있는 바비큐 시설도 마련되어 있습니다. 바닷가앞펜션에서 에메랄드빛 제주 바다가 정원처럼 펼쳐지는 곳에서 제주 여행의 시작과 끝을 함께 하시길 바랍니다.",

    "위치정보": "주소: 제주특별자치도 제주시 애월읍 해안로 1234, 교통편: 제주공항에서 차로 약 1시간, 주변 관광지:, 애월항 3.79km, 이호테우 해수욕장 10.70km, 도두항 13.16km, 제주 환상길 산책로 펜션 앞",

    "시설정보": "오션뷰 객실, 해수욕장 근처 (3.79km 인근 애월항), 노래방 기기 대여 가능, 테라스 흡연 가능, 펜션 내 주차 공간 제공 (최대 12대 주차 가능), 호텔식 화이트 침구류 구비, 야외 테라스 및 바비큐 시설 제공",

    "예약정보": "예약 인원 초과 시 사전 연락 필수, 기준 인원 초과 시 추가 비용 발생, 최대 인원 초과 시 입실 불가 및 환불 불가, 반려동물 입실 불가 (해당 사유로 인한 환불 불가), 청소년 혼숙 금지 및 미성년자 예약 제한, 입실 및 퇴실 시간 준수 필요, 실시간 예약 시 중복 예약 발생 가능성 있음 (중복 예약 시 전액 환불)",

    "서비스정보": "4인 이상 예약 시 흑돼지 갈매기살 600g 무료 제공, 1층 식당에서 주류(소주, 맥주 등) 및 음료 판매, 투숙객 객실에서 노래방 기기 대여 가능 (사전 예약 필수), 야외에서 일몰 및 야경 감상 가능, 감사한 마음으로 최선을 다하는 서비스 제공",

    "환불규정" : "기준 인원 초과 시 추가 인원 비용 발생, 최대 인원 초과 시 입실 불가 및 환불 불가, 반려동물 동반 시 입실 거부 및 환불 불가, 청소년 혼숙 금지 및 미성년자 예약 시 환불 불가",

    "가변정보": None,

    "특이정보": "22시 이후 입실 시 펜션으로 미리 연락 필요, 객실 내 흡연 금지 및 지정된 장소에서 흡연, 무분별한 오락 음주 고성방가 자제, 객실 내 직화 방식 요리 및 개인 취사도구 반입 금지, 시설물 훼손 및 분실 시 투숙객 책임, 요금 및 요금표 오류 시 예약 자동 취소 및 정상 금액으로 재예약 가능, 2024년 3월 29일부터 일부 숙소는 1회용품(칫솔, 면도기 등) 무료 제공 불가(자원의 절약과 재활용 촉진에 관한 법률 일부 개정에 따름)"

}

In [283]:
from openai import OpenAI # openai==1.2.0

client = OpenAI(
    api_key="up_TQpptc9W6GuPYyQ1o2rIYpgMiY6UN",
    base_url="https://api.upstage.ai/v1/solar"
)

In [284]:
messages = [
    # 메세지 로그
    {
        "role": "system",
        "content": "너는 숙박 업소 챗봇이야. 고객들에게 젠틀하게 서비스를 제공하도록 해."
    }
]

In [285]:
formatter_log = [
    # 포맷 로그
    {
        "role": "system",
        "content": "너는 숙박 업소 챗봇이야. 고객들들에게 최선을 다해 서비스를 제공하도록 해. 너는 내가 주는 정보를 고객들에게 잘 표현하여 전달하는 역할이야. 내가 주는 딱딱한 정보들을 고객들이 만족할 수는 서비스로 표현을 만들어서 제공하도록 해."
    }


]

In [294]:
# Function Calling을 위한 함수들 목록
tools = [
        {
            "type": "function",
            "function": {
                "name": "use_data",
                "description": "고객이 숙박시설에 대한 분위기 등의 소개를 물어보면 '기본정보', 숙박시설의 위치를 물어보면 '위치정보', 숙박시설 내부 시설 내용을 물어보면 '시설정보', 예약과 관련한 내용을 물어보면 '예약정보', 환불관련 내용을 물어보면 '환불규정', 숙박시설에서 제공하고 있는 서비스에 대해 물어보면 '서비스정보', 식사메뉴와 같이 때에 따라 달라질 수 있는 정보들을 물어보면 '가변정보'로 분류해줘. 그 외 숙소관련 특이정보는 '특이정보'로 분류해줘. 숙박업소 챗봇 기준으로 해당 분류에 적합하지 않다고 생각하는 것은 분류에 넣지마.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "category": {
                            "type": "string",
                            "description": "기본정보, 위치정보, 시설정보, 예약정보, 서비스정보, 가변정보",
                        },
                        "category": {"type": "string", "enum": ["기본정보", "위치정보", "시설정보", "예약정보", "서비스정보", "가변정보", "특이정보", "환불규정"]},
                    },
                    "required": ["category"],
                },
            },
        }


    ]

In [287]:
def formatter(category, req):

  category_data = data[category]

  content = f"{category_data}, 이 정보를 읽어. 고객님은 {req}와 관련한 정보를 요청하셨어. 정보 안에서만 고객에게 유용한 정보를 제공해. 해당 내용에 없는 정보는 절대로 함부로 제공을 해서는 안돼. 이 점을 꼭 기억해야해."

  if category_data is None:
    content = "해당 정보가 없습니다."

  formatter_log.append({
        "role": "user",
        "content": content
    })

  response = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=formatter_log,
        stream=False

  )

  full_response = response.choices[0].message.content

  return full_response  # 최종 응답을 반환

In [288]:
def chat(req):
    # 사용자 메시지를 추가하는 함수
    messages.append({
        "role": "user",
        "content": req
    })

    # 새로운 stream 객체 생성
    response = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=messages,
        stream=False,
        tools=tools,
        tool_choice="auto",
    )


    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls


    if tool_calls:
        for tool_call in tool_calls:
            # 인자들 확인
            function_args = json.loads(tool_call.function.arguments)
            selected_category = function_args.get("category")

            # 선택된 category를 기반으로 응답 생성
            tool_response = formatter(selected_category, req)
            messages.append({
                "role": "assistant",
                "content": tool_response,
                "category": selected_category
            })
    else:
        # tool_choice가 없으면 일반적인 모델 응답을 추가
        messages.append({
            "role": "assistant",
            "content": response.choices[0].message.content
        })

    return messages[-1]["content"]



In [289]:
print(chat("환불을 하고 싶어"))

고객님, 저희는 환불 정책에 따라 다음과 같은 경우에 환불이 불가능합니다:

1. 기준 인원을 초과한 추가 인원이 발생한 경우
2. 최대 인원을 초과한 경우
3. 반려동물을 동반한 경우
4. 청소년 혼숙이 발생한 경우
5. 미성년자가 예약한 경우

만약 환불을 원하신다면, 예약 취소 및 환불 정책을 참고하시어 해당 기간 내에 취소 요청을 해주시기 바랍니다. 예약 취소 및 환불 정책은 예약 시 안내되었거나, 예약 사이트 또는 숙박 시설의 정책에 명시되어 있을 수 있습니다.


In [290]:
formatter_log

[{'role': 'system',
  'content': '너는 숙박 업소 챗봇이야. 고객들들에게 최선을 다해 서비스를 제공하도록 해. 너는 내가 주는 정보를 고객들에게 잘 표현하여 전달하는 역할이야. 내가 주는 딱딱한 정보들을 고객들이 만족할 수는 서비스로 표현을 만들어서 제공하도록 해.'},
 {'role': 'user',
  'content': '기준 인원 초과 시 추가 인원 비용 발생, 최대 인원 초과 시 입실 불가 및 환불 불가, 반려동물 동반 시 입실 거부 및 환불 불가, 청소년 혼숙 금지 및 미성년자 예약 시 환불 불가, 이 정보를 읽어. 고객님은 환불을 하고 싶어와 관련한 정보를 요청하셨어. 정보 안에서만 고객에게 유용한 정보를 제공해. 해당 내용에 없는 정보는 절대로 함부로 제공을 해서는 안돼. 이 점을 꼭 기억해야해.'}]

In [293]:
messages

[{'role': 'system', 'content': '너는 숙박 업소 챗봇이야. 고객들에게 젠틀하게 서비스를 제공하도록 해.'},
 {'role': 'user', 'content': '환불을 하고 싶어'},
 {'role': 'assistant',
  'content': '고객님, 저희는 환불 정책에 따라 다음과 같은 경우에 환불이 불가능합니다:\n\n1. 기준 인원을 초과한 추가 인원이 발생한 경우\n2. 최대 인원을 초과한 경우\n3. 반려동물을 동반한 경우\n4. 청소년 혼숙이 발생한 경우\n5. 미성년자가 예약한 경우\n\n만약 환불을 원하신다면, 예약 취소 및 환불 정책을 참고하시어 해당 기간 내에 취소 요청을 해주시기 바랍니다. 예약 취소 및 환불 정책은 예약 시 안내되었거나, 예약 사이트 또는 숙박 시설의 정책에 명시되어 있을 수 있습니다.',
  'category': '환불규정'},
 {'role': 'user', 'content': '예약하려고 하는데 반려동물을 데리고 들어갈 수 있어?'},
 {'role': 'assistant',
  'content': '고객님, 저희 숙박 시설은 반려동물 동반이 가능한 시설입니다. 그러나 반려동물을 동반하실 경우, 추가적인 요금이나 제한 사항이 있을 수 있습니다. 자세한 내용은 예약 시 문의해주시기 바랍니다.'}]

In [295]:
print(chat("예약하려고 하는데 반려동물을 데리고 들어갈 수 있어?"))

고객님, 저희 숙박 시설은 반려동물 동반이 가능한 시설입니다. 그러나 반려동물을 동반하실 경우, 추가적인 요금이나 제한 사항이 있을 수 있습니다. 자세한 내용은 예약 시 문의해주시기 바랍니다.
